# `eGSIM` request (Residuals)

In [4]:
# define imports:
import warnings; warnings.filterwarnings("ignore")
import os
import io
import json
from typing import List, Dict, Tuple, Optional, Union
# external libraries (pip install ...):
import requests
import numpy as np  # numeric python (faster numeric array manipoulation)
import pandas as pd  # handling tabular data efficiently (e.g. pd.DataFrame)
import tables  # required by pandas only if you download data in hdf instead of csv
import matplotlib.pyplot as plt  # plotting library

# Jupyter notebook stuff (remove if you are coding from a .py file):
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML
# make wide cells display a scrollbar automatically:
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))
from IPython.core.display import HTML
# make tables cells with a border:
display(HTML("<style>th, td{border: 1px solid #DDE !important;}</style>"))


# request function:
def get_residuals_from_egsim(
    model: List,
    imt: List,
    flatfile: Union[io.IOBase, str],
    query_string=None,
    likelihood=False
) -> pd.DataFrame:
    """Retreive the residuals for the flatfile and the selected
    set of ground motion models and intensity measure types

    Args:
        flatfile: A string denoting a predefined flatifle (e.g. "esm2018"), or a 
            [file object](https://docs.python.org/3/glossary.html#term-file-object), e.g.:
            ```with open(flatfile_path, "rb") as ff:
                   get_residuals_from_egsim(... flatfile=ff...)
            ```
        model: ground motion model(s) (OpenQuake class names)
        imt: intensity measure type(s) (e.g. PGA, PGV, SA(0.1) etc.)
        query_string: Selection query to apply to the data (e.g. "mag>6")
        likelihood: bool (default False): compute the residuals likelihood 
            according to [Scherbaum et al. (2004)](https://doi.org/10.1785/0120030147)
    Returns:
        a pandas DataFrame with the flatfile records and their computed residuals
    """ 
    
    # request parameters:
    parameters = {
        'model': model,
        'imt': imt,
        'format': 'hdf' # you can change this to csv (e.g., tables is not installed), but hdf should be more performant
    }
    if query_string:
        parameters['data-query'] = query_string
    
    # arguments to be passed to `requests.post`. Some of them will be filled depending 
    # on the request type (see if below):
    args = {'data':None, 'json':None, 'files':None}
    
    if isinstance(flatfile, str):
        parameters['flatfile'] = flatfile
        args['json'] = parameters
    else:
        args['data'] = parameters
        args['files'] = {'flatfile': flatfile}

    # POST request for eGSIM
    response = requests.post(
        "{{ url }}",  # the base request URL for residuals
        **args
    )
    response.raise_for_status() # if server response code denote unsuccessful response, fallbak below

    # `response.content` is a bytes sequence (in-memory file) representing the
    # result DataFrame, either in CSV or HDF format. Read the conent into a DataFrame:
    
    if parameters['format'] == 'hdf':
        # `pd.read_hdf` works for HDF files on disk. Workaround:
        with pd.HDFStore(
                "data.h5",  # apparently unused for in-memory data
                mode="r",
                driver="H5FD_CORE",  # create in-memory file
                driver_core_backing_store=0,  # for safety, just in case
                driver_core_image=response.content) as store:
            dframe = store[list(store.keys())[0]]
    else:
        # use `pd.read_csv` with a BytesIO (file-like object) as input: 
        dframe = pd.read_csv(io.BytesIO(response.content), header=[0, 1, 2], index_col=0)
        
    return dframe

# define request parameters:
model = {{ model }};
imt = {{ imt }};
{% if uploaded_flatfile %}
flatfile_path = ""  # <- type HERE the flatfile path (CSV file)
if not os.path.isfile(flatfile_path):
    raise FileNotFoundError('`flatfile_path` must denote a path to an existing file in CSV format')
{% else %}
flatfile = {{ flatfile }};
{% endif %}
likelihood = {{ likelihood }};
query_string {{ query_string }};

# query residuals:
try:
    {% if uploaded_flatfile %}
    with open(flatfile_path, "rb") as flatfile:
        residuals = get_residuals_from_egsim(model, imt, flatfile, query_string, likelihood)
    {% else %}
    residuals = get_residuals_from_egsim(model, imt, flatfile, query_string, likelihood)
    {% endif %}
except requests.exceptions.HTTPError as exc:
    # client or server error, print eGSIM error message:
    code = exc.response.status_code
    msg = response.json()['message']
    print(f"HTTPError (code={code}): {msg}")

# show residuals dataframe:
display(residuals)

# to save results on disk:
# residuals.to_hdf(...)
# residuals.to_csv(...)
# See the official doc for more details

SyntaxError: invalid syntax (40950391.py, line 101)